# 15 Implementing Scaled Dot-Product Attention in Keras

In [31]:
from keras.backend import softmax
from tensorflow import cast, float32, math, matmul
from tensorflow.keras.layers import Layer
from numpy import random

random.seed(42)

## 15.1 Recap of the Transformer Architecture

The encoder and the decoder share much of their architecture. At the heart of their numerous, stacked, multi-head attention blocks is the *scaled dot-product attention* mechanism.
In the multi-head attention block of the encoder, the query, key and value vectors (which form the Query, Key and Value matrices once concatenated) are simply the encoded and embedded (see ch. 14) input sequence. Similarly, on the decoder side, the first attention block gets the encoded/embedded _target_ sequence in the form of query, key and value vectors. However, the _second_ attention block receives the final output of the encoder block for its keys and values but uses the [normalized] output of its own first attention block as its queries. (The latter can be thought of as the decoder output from the "previous time step", but do keep in mind that there is no recurrence here and everything is fed to the model all at once). 
We will denote the dimensionality of queries and keys with $d_k$ and that of values with $d_v$.
First we calculate the matrix multiplication of $Q$ and $K^T$ (which is equivalent to calculating the dot products of query and key _vectors_). Then we scale the result by the square root of $d_k$ to get the _attention scores_. We feed the result to the $softmax$ function to get _attention weights_. And finally, we scale the the value vectors by matrix-multiplying the result with $V$.
$$attention(Q, K, V) = softmax\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

A "mask" can optionally be applied to the attention scores before they are fed to the $softmax$ function. Here are two conceivable applications for this:  
- A "look-ahead mask" (as in the first attention block of the decoder) can prevent the model from, you guessed it, "looking ahead" and attending to succeeding tokens in the target sequence. ("Succeeding" in the sense that it has not yet reached them and output(ted) a prediction for those positions in the target sequence).
- A "padding mask" can prevent the padding (often zero) tokens from being processed along with meaningful tokens both in the encoder and decoder stages.
Masking works by replacing the attention scores to be masked with $-\infty$ so that $softmax$ will result in zeros for those positions.

## 15.2 Implementing the Scaled Dot-Product Attention from Scratch

In [32]:
# Implementing the Scaled Dot Product Attention
class DotProductAttention(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, queries, keys, values, d_k, mask=None):
        # Score the queries against the keys after transposing the latter, and then scale
        scores = matmul(queries, keys, transpose_b=True) / math.sqrt(cast(d_k, float32))
        # Apply mask to the attention scores
        if mask is not None:
            scores += float("-inf") * mask
        # Compute the weights using a softmax operation
        weights = softmax(scores)
        # Compute attention by a weighted sum of the value vectors
        return matmul(weights, values)

## 15.3 Testing Out the Code

In [33]:
d_k = 64 # Dimensionality of the linearly projected queries and keys
d_v = 64 # Dimensionality of the linearly projected values
batch_size = 64 # Batch size from the training process

# Dummy data follows...
# In reality, these would be obtained from the tokenized and then embedded sequences.
input_seq_length = 5 # Maximum length of the input sequence
queries = random.random((batch_size, input_seq_length, d_k))
keys = random.random((batch_size, input_seq_length, d_k))
values = random.random((batch_size, input_seq_length, d_v))

attention = DotProductAttention()
print(attention(queries, keys, values, d_k))

tf.Tensor(
[[[0.42829984 0.5291363  0.48467714 ... 0.60236514 0.6314437  0.3679649 ]
  [0.42059594 0.51898783 0.46809807 ... 0.5975176  0.63140476 0.3960448 ]
  [0.4529176  0.53372943 0.482216   ... 0.5861657  0.6170542  0.35611773]
  [0.4353886  0.52972203 0.4782614  ... 0.5917442  0.62593013 0.3666562 ]
  [0.42998835 0.51891106 0.481131   ... 0.610327   0.63044834 0.39192218]]

 [[0.61051536 0.5024951  0.401304   ... 0.7148773  0.36341453 0.5512419 ]
  [0.5842008  0.5239525  0.4311911  ... 0.72335523 0.36001056 0.5697574 ]
  [0.56449413 0.559814   0.4412013  ... 0.6975891  0.34060013 0.57147545]
  [0.5878388  0.52120656 0.42275843 ... 0.7043989  0.34812245 0.556117  ]
  [0.5880349  0.52016133 0.43390357 ... 0.7050327  0.35547623 0.5617097 ]]

 [[0.42078587 0.49814874 0.49267095 ... 0.49847665 0.4983842  0.2441965 ]
  [0.4155558  0.4971297  0.49442884 ... 0.5093836  0.5085627  0.24680805]
  [0.42648807 0.4867728  0.47802505 ... 0.49915406 0.4939838  0.24511527]
  [0.42818245 0.4891822

**Note:** The output shape is `(batch size, sequence length, dim_values)`.